In [1]:
import sys
sys.path.append('../')
from scripts.helpers import ROC_fromTuples
from scripts.selection import *
import ROOT
from ROOT import TTree 
from sklearn.metrics import roc_curve, auc
import numpy as np
import uproot
import awkward as ak
%matplotlib inline
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.22/02


In [3]:
# get trees for efficiency from file
fileName_eff = "/Users/mascella/workspace/EPR-workspace/analysis_deepTau/data/VBFToTauTau_minPt20.root"
treeName_gen = "gen_counter" 
treeName_in = "initial_counter" 
events_gen, events_in = getEvents_fromFile(fileName_eff, treeName_gen, treeName_in)
taus = getTaus(events_in)
print(taus.pt[range(3)])
L1taus = getL1taus(events_in)
taus = L1THLTTauMatching(L1taus, taus)
taus = HLTJetPairDzMatchFilter(taus)
print(taus.pt[range(3)])
print(true_tau_selection(taus)[range(3)])
print(gen_tau_selection(taus)[range(3)])
print(reco_tau_selection(taus, minPt=20., eta_sel=False)[range(3)])
num_tau_mask = true_tau_selection(taus) & gen_tau_selection(taus) & reco_tau_selection(taus, minPt=20., eta_sel=False)
print(num_tau_mask[range(3)])
# L1taus = getL1taus(events_in)
# L2taus = L1THLTTauMatching(L1taus, taus)
# L2taus = HLTJetPairDzMatchFilter(L2taus)
# L2taus = L2taus[iso_tau_selection(L2taus, "tightIsoAbs", "tightIsoRel")]
# print(L2taus.flatten().size)
# L2taus = L2taus[(L2taus["tightIsoAbs"]==1) | (L2taus["tightIsoRel"]==1)]
# print(L2taus.flatten().size)

[[177.94908 116.30658 4.5636206 4.4635415 1.6282612] [68.61354 45.64627 36.557236] [102.1446 56.470238 54.399242 ... 0.99585944 0.9592106 0.6880077]]
[[177.94908 116.30658] [68.61354 45.64627 36.557236] [102.1446 56.470238 54.399242]]
[[False True] [False True True] [False True False]]
[[False True] [False True True] [False True True]]
[[True True] [True True True] [True True True]]
[[False True] [False True True] [False True False]]


In [8]:
# print(L2taus.deepTau_VSjet[(L2taus.lepton_gen_match==5) | (L2taus.lepton_gen_match==6)].flatten().size)
gen_match = L2taus.lepton_gen_match # generator info
pred_all = L2taus.deepTau_VSjet
truth_all = where(gen_match==5, 1, 0)
sel_tauorjets = (L2taus.lepton_gen_match==5) | (L2taus.lepton_gen_match==6)
pred = pred_all[sel_tauorjets] # deepTau predictions for tau_h and jets
truth = truth_all[sel_tauorjets] # truth info for tau_h and jets
print(truth.flatten().size)
truth = truth[pred==0]
print(truth.flatten().size)
pred = pred[pred==0]

fpr, tpr, thr = roc_curve(truth.flatten(), pred.flatten())

53011
90


In [ ]:
taus = ak.JaggedArray.zip(e=events.tau_e, pt=events.tau_pt, eta=events.tau_eta, phi=events.tau_phi, 
                                looseIsoAbs=events.tau_looseIsoAbs, looseIsoRel=events.tau_looseIsoRel,
                                mediumIsoAbs=events.tau_looseIsoAbs, mediumIsoRel=events.tau_looseIsoRel,
                                tightIsoAbs=events.tau_tightIsoAbs, tightIsoRel=events.tau_tightIsoRel,
                                gen_e = events.gen_tau_e, gen_pt=events.gen_tau_pt, gen_eta=events.gen_tau_eta, gen_phi=events.gen_tau_phi,
                                lepton_gen_match=events.lepton_gen_match)
# L1tau = ak.JaggedArray.zip(pt=events_in.L1tau_pt, eta=events_in.L1tau_eta, phi=events_in.L1tau_phi)
taus.pt

In [ ]:

# true_taus = events_in[events_in["lepton_gen_match"]==5]
# print(true_taus["lepton_gen_match"][0,5])
# true_jets = events_in[events_in["lepton_gen_match"]==6]
# print(true_jets["lepton_gen_match"][0,5])
r = range(5)
var_abs = "tau_mediumIsoAbs"
var_rel = "tau_mediumIsoRel"
cut = (events_in[var_abs]>0) | (events_in[var_rel]>0)
print((events_in[var_abs]>0)[r])
print((events_in[var_rel]>0)[r])
print(cut[r])

tau_sel = cut & (events_in["lepton_gen_match"]==5)
jet_sel = cut & (events_in["lepton_gen_match"]==6)
print((events_in["lepton_gen_match"])[r])
print(tau_sel[r])
print(tau_sel.flatten()[range(10)])
print((events_in["lepton_gen_match"]==5).flatten()[range(10)])
tpr = (tau_sel.flatten().sum())/((events_in["lepton_gen_match"]==5).flatten().sum())
fpr = (jet_sel.flatten().sum())/((events_in["lepton_gen_match"]==6).flatten().sum())
print(tpr)
print(fpr)

# tpr = ((true_taus[cut_tau]["lepton_gen_match"]).counts.sum())/(true_taus["lepton_gen_match"].counts.sum())
# fpr = ((true_jets[cut_jet]["lepton_gen_match"]).counts.sum())/(true_jets["lepton_gen_match"].counts.sum())


In [ ]:
plt.hist(events_in['gen_tau_eta'][events_in['lepton_gen_match']==5].flatten(), bins=50, range=[0, 3])

In [ ]:
r = range(2,5)
# print((events_gen['lepton_gen_match']==1).sum().sum())
# plt.hist(events_gen['lepton_gen_match'].flatten())

# print(((events_gen['gen_tau_pt']>0).sum()>2).sum())
# print(events_in["lepton_gen_match"][r])
# print(events_in['gen_tau_pt'][r]) 
# print(events_in['tau_pt'][r])
# mask = (events_in["lepton_gen_match"]==5) & (events_in['gen_tau_pt']>35.) & (events_in['tau_pt']>40.)
# print(mask[r])
# true_taus_pred = events_in['deepTau_VSjet'][mask] # deepTau prediction for tau_h
# print(events_in['deepTau_VSjet'][r])
# print(true_taus_pred[r])
# plt.hist((events_in['gen_tau_pt'][mask]).flatten(), bins=50, range=[20, 100])
gen_minPt = 35
gen_maxEta = 2.1
print(len(events_gen))
print(events_gen['gen_tau_pt'][r])
print(events_gen['gen_tau_eta'][r])
mask_den = (events_gen['gen_tau_pt']>gen_minPt) & (events_gen['gen_tau_eta']<gen_maxEta) & (events_gen['gen_tau_eta']>-gen_maxEta)
print(mask_den[r])
print(events_gen['gen_tau_pt'][mask_den].counts[r])
print((events_gen['gen_tau_pt'][mask_den].counts>=2).sum())


In [ ]:
# generate ROC curve for deepTau_VSjet
fpr, tpr, thr, pred, truth = ROC_fromTuples(events_in)
score = auc(fpr, tpr)
print("AUC ROC:", score)
plt.yscale('log')
plt.xlabel(r'$\tau$ ID efficiency')
plt.ylabel('jet misID probability')
plt.plot(tpr, fpr, '-')
print(len(thr))

In [ ]:
# deepTau_VSjet distribution for signal and background
gen_match = events_in["lepton_gen_match"] # generator info
pred_all = events_in["deepTau_VSjet"]
truth_all = where(gen_match==5, 1, 0)
sel_tauorjets = where((gen_match==5)|(gen_match==6), 1, 0)>0
pred = pred_all[sel_tauorjets] # deepTau predictions for tau_h and jets
truth = truth_all[sel_tauorjets] # truth info for tau_h and jets
plt.hist(pred[truth==1].flatten(), range=[0,1.25], bins=50, alpha=0.5, label=r"$\tau_h$")
plt.hist(pred[truth==0].flatten(), bins=50, range=[0,1.25], alpha=0.5, label="jets")
plt.legend()
plt.xlabel("deepTau_VSjet")
plt.savefig("VSjet_distr.pdf")

In [ ]:
# compute efficiencies
# from scipy.stats import loguniform
# a, b = 0.01, 1
# x = np.linspace(loguniform.ppf(0.99, a, b),
#                 loguniform.ppf(0.01, a, b), 100)
# thr = loguniform.pdf(1-x, a, b)
eff = compute_efficiency(events_in, len(events_gen), thr, 40.)
print(eff)
print(len(events_gen))

In [ ]:
# get trees for rates from file
fileName_rates = "/Users/mascella/workspace/EPR-workspace/analysis_deepTau/data/HLTPhys_1-8.root"
events_gen_rates, events_in_rates = getEvents_fromFile(fileName_rates, treeName_gen, treeName_in)
print(len(events_in_rates))
plt.hist(events_in_rates['deepTau_VSjet'].flatten(), bins=50, alpha=0.5)
plt.hist(events_in_rates['tau_pt'].flatten(), bins=50)

In [ ]:
# compute rates
Ntot = len(events_gen_rates)
L1rate = 73455.34
rates = compute_rates(events_in_rates, Ntot, L1rate, thr, 40.)
print(rates)

In [ ]:
plt.xlabel("Efficiency")
plt.ylabel("Rate [Hz]")
plt.title("Rate vs Efficiency")
plt.plot(eff[range(0, len(thr)-1000, 100)], rates[range(0, len(thr)-1000, 100)], '.-')
print(rates[range(0, len(thr)-1000, 100)])